In [3]:
!pip install opencv-python mediapipe scikit-learn pandas


In [2]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from IPython.display import display, clear_output


In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

columns = ['label'] + [f'{i}_{axis}' for i in range(21) for axis in ['x', 'y', 'z']]
df = pd.DataFrame(columns=columns)
df.to_csv('sign_data.csv', index=False)


In [ ]:
cap = cv2.VideoCapture(0)
print("Press A–Z while showing sign. ESC to stop.")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            key = cv2.waitKey(10) & 0xFF
            if 65 <= key <= 90:  # A-Z
                row = [chr(key)]
                for lm in hand_landmarks.landmark:
                    row.extend([lm.x, lm.y, lm.z])
                with open('sign_data.csv', 'a', newline='') as f:
                    pd.DataFrame([row]).to_csv(f, header=False, index=False)
                print(f"✅ Saved for {chr(key)}")

    cv2.imshow("Collecting Data", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


Press A–Z while showing sign. ESC to stop.


In [ ]:
df = pd.read_csv('sign_data.csv')
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestClassifier()
model.fit(X_train, y_train)

print("🎯 Accuracy:", model.score(X_test, y_test))


In [ ]:
with open('sign_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("✅ Model saved to sign_model.pkl")


In [ ]:
with open('sign_model.pkl', 'rb') as f:
    model = pickle.load(f)

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()


In [ ]:
for i in range(100):  # Capture 100 frames
    ret, frame = cap.read()
    if not ret:
        continue

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            data = []
            for lm in hand_landmarks.landmark:
                data += [lm.x, lm.y, lm.z]
            prediction = model.predict([data])[0]
            cv2.putText(frame, f"Prediction: {prediction}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

    # Show inside notebook
    clear_output(wait=True)
    from PIL import Image
    display(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

cap.release()
cv2.destroyAllWindows()


In [ ]:
import pandas as pd

try:
    df = pd.read_csv("sign_data.csv")
    print("✅ CSV found! Shape:", df.shape)
    display(df.head())
except FileNotFoundError:
    print("❌ CSV file not found. Please collect data first.")
